In [ ]:
async def finance_agent_generator(query: str):
    graph = create_finance_agent_graph()
    initial_state = AgentState(
        query=query,
        processed_query="",
        use_graph_db=False,
        graph_db_result=[],
        expanded_queries=[],
        subgraphs_to_execute=3,
        subgraph_results={},
        final_response=""
    )
    current_state = initial_state
    
    last_node = None
    async for chunk in graph.astream(current_state, stream_mode="updates"):
        for node, node_state in chunk.items():
            if isinstance(node_state, dict):
                yield f"data: {json.dumps({'type': 'step', 'content': node})}\n\n"
                last_node = node
            else:
                print(f"Unexpected state type for node {node}: {type(node_state)}.")
        
        # After processing all nodes, check if we have a final response
        if last_node and 'final_response' in chunk[last_node]:
            final_response = chunk[last_node]['final_response']
            if final_response:
                yield f"data: {json.dumps({'type': 'final', 'content': final_response})}\n\n"

    yield "data: [DONE]\n\n"

In [ ]:
  const handleAction = useCallback(async (userInput) => {
    setIsProcessing(true);
    setError(null);
    setStatusMessage(null);
    setProcessingSteps([]);
    
    setChatMessages(prevMessages => [...prevMessages, { role: "user", content: userInput }]);
    
    try {
      const content = await getEmailContent();
      const payload = {
        userId: userConfig.userId,
        emailContent: content,
        prompt: userInput,
      };

      const response = await fetch("http://localhost:8001/fluent_ai", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify(payload),
      });

      if (!response.ok) throw new Error(`HTTP error! status: ${response.status}`);
      
      const reader = response.body.getReader();
      const decoder = new TextDecoder();

      let buffer = '';
      while (true) {
        const { value, done } = await reader.read();
        if (done) break;
        
        buffer += decoder.decode(value, { stream: true });
        const lines = buffer.split('\n');
        buffer = lines.pop();

        for (const line of lines) {
          if (line.startsWith('data: ')) {
            try {
              const data = JSON.parse(line.slice(5));
              if (data === '[DONE]') {
                setIsProcessing(false);
              } else {
                handleUpdate(data);
              }
            } catch (error) {
              console.error('Error parsing SSE data:', error);
            }
          }
        }
      }
    } catch (e) {
      console.error(`Error in handleAction: ${e.message}`);
      setError('Failed to process your request. Please try again.');
      setIsProcessing(false);
    }
  }, [userConfig, getEmailContent]);

  const handleUpdate = useCallback((data) => {
    if (data.type === 'step') {
      setProcessingSteps(prevSteps => [...prevSteps, data.content]);
    } else if (data.type === 'final') {
      setChatMessages(prevMessages => [
        ...prevMessages,
        { role: "assistant", content: wrapInHtml(data.content) }
      ]);
      setIsProcessing(false);
    }
  }, []);